In [1]:
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from deep_audio import Directory, JSON

In [2]:
method_algo = 'lpcc'
n_rate = 24000

DATASET_PATH = f'processed/{method_algo}/{method_algo}_{n_rate}.json'

inputs, targets, mapping = Directory.load_json_data(DATASET_PATH, inputs_fieldname=method_algo)

In [3]:
def create_dense_model():
    # build the network architecture
    model = keras.Sequential([
        # input layer
        keras.layers.Flatten(input_shape=(inputs.shape[1], inputs.shape[2])),

        # 1st hidden layer
        keras.layers.Dense(512, activation='relu'),

        # 2nd hidden layer
        keras.layers.Dense(256, activation='relu'),

        # 3rd hidden layer
        keras.layers.Dense(128, activation='relu'),

        # output layer
        keras.layers.Dense(len(mapping), activation='softmax'),
    ])

    return model

In [4]:
# SPLIT DOS DADOS
random_state = 42
# for random_state in [5438, 53, 14]:
#     for _ in range(4):
# split data into train and test set
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs,
                                                                          targets,
                                                                          test_size=0.2,
                                                                          stratify=targets,
                                                                          random_state=random_state)

inputs_train, inputs_valid, targets_train, targets_valid = train_test_split(inputs_train,
                                                                          targets_train,
                                                                          test_size=0.2,
                                                                          stratify=targets_train,
                                                                          random_state=random_state)


In [5]:
# SALVA UMA CÓPIA DOS DADOS DE TESTE

# data = {
#     "mapping": mapping,
#     "labels": targets_test,
#     "mfcc": inputs_test,
# }

# JSON.create_json_file(f'datatest/deep/datatest_{random_state}_{inputs.shape[0]}.json', data)

In [6]:
# CRIA O MODELO
model = create_dense_model()

# COMPILA A REDE
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])

In [7]:
# SALVA A ESTRUTURA DO MODELO

timestamp = int(time.time())

Directory.create_directory(f'models/{method_algo}/{timestamp}')

JSON.create_json_file(f'models/{method_algo}/{timestamp}/model_structure.json', model.to_json())

model_save_filename = f'models/{method_algo}/{timestamp}/model_weight.h5'

# DECIDE QUANDO PARAR
earlystopping_cb = keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)

# SALVA OS PESOS
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
model_save_filename, monitor="val_accuracy", save_best_only=True
)

In [8]:
# TREINA O MODELO
history = model.fit(inputs_train, targets_train,
            validation_data=(inputs_valid, targets_valid),
            epochs=10000,
            batch_size=128,
            callbacks=[earlystopping_cb, mdlcheckpoint_cb])

Epoch 1/10000
89/89 [==============================] - 6s 64ms/step - loss: 44.1801 - accuracy: 0.0079 - val_loss: 4.7272 - val_accuracy: 0.0145
Epoch 2/10000
89/89 [==============================] - 4s 44ms/step - loss: 4.7017 - accuracy: 0.0114 - val_loss: 4.7108 - val_accuracy: 0.0131
Epoch 3/10000
89/89 [==============================] - 3s 38ms/step - loss: 4.6905 - accuracy: 0.0121 - val_loss: 4.7117 - val_accuracy: 0.0127
Epoch 4/10000
89/89 [==============================] - 3s 39ms/step - loss: 4.6814 - accuracy: 0.0134 - val_loss: 4.7139 - val_accuracy: 0.0127
Epoch 5/10000
89/89 [==============================] - 3s 39ms/step - loss: 4.6787 - accuracy: 0.0134 - val_loss: 4.7116 - val_accuracy: 0.0138
Epoch 6/10000
89/89 [==============================] - 3s 38ms/step - loss: 4.6737 - accuracy: 0.0135 - val_loss: 4.7160 - val_accuracy: 0.0134
Epoch 7/10000
89/89 [==============================] - 3s 38ms/step - loss: 4.6704 - accuracy: 0.0147 - val_loss: 4.7147 - val_accuracy

89/89 [==============================] - 3s 39ms/step - loss: 3.8853 - accuracy: 0.1078 - val_loss: 5.4451 - val_accuracy: 0.0106
Epoch 58/10000
89/89 [==============================] - 3s 39ms/step - loss: 3.9146 - accuracy: 0.1033 - val_loss: 5.4573 - val_accuracy: 0.0134
Epoch 59/10000
89/89 [==============================] - 3s 39ms/step - loss: 3.8516 - accuracy: 0.1104 - val_loss: 5.5024 - val_accuracy: 0.0173
Epoch 60/10000
89/89 [==============================] - 3s 39ms/step - loss: 3.8375 - accuracy: 0.1144 - val_loss: 5.4706 - val_accuracy: 0.0134
Epoch 61/10000
89/89 [==============================] - 3s 39ms/step - loss: 3.8235 - accuracy: 0.1218 - val_loss: 5.5790 - val_accuracy: 0.0191
Epoch 62/10000
89/89 [==============================] - 3s 39ms/step - loss: 3.7746 - accuracy: 0.1262 - val_loss: 5.6551 - val_accuracy: 0.0191
Epoch 63/10000
89/89 [==============================] - 3s 39ms/step - loss: 3.7590 - accuracy: 0.1246 - val_loss: 5.5839 - val_accuracy: 0.0187


89/89 [==============================] - 3s 39ms/step - loss: 2.4380 - accuracy: 0.3885 - val_loss: 8.5868 - val_accuracy: 0.0230
Epoch 114/10000
89/89 [==============================] - 3s 39ms/step - loss: 2.3724 - accuracy: 0.4035 - val_loss: 8.6840 - val_accuracy: 0.0191
Epoch 115/10000
89/89 [==============================] - 4s 39ms/step - loss: 2.3615 - accuracy: 0.4045 - val_loss: 9.5815 - val_accuracy: 0.0212
Epoch 116/10000
89/89 [==============================] - 3s 39ms/step - loss: 2.3213 - accuracy: 0.4124 - val_loss: 9.0162 - val_accuracy: 0.0166
Epoch 117/10000
89/89 [==============================] - 5s 52ms/step - loss: 2.3542 - accuracy: 0.4097 - val_loss: 9.2791 - val_accuracy: 0.0212
Epoch 118/10000
89/89 [==============================] - 4s 44ms/step - loss: 2.3138 - accuracy: 0.4204 - val_loss: 9.3956 - val_accuracy: 0.0237
Epoch 119/10000
89/89 [==============================] - 4s 42ms/step - loss: 2.2911 - accuracy: 0.4230 - val_loss: 8.9394 - val_accuracy: 0

Epoch 169/10000
89/89 [==============================] - 3s 39ms/step - loss: 1.4985 - accuracy: 0.5998 - val_loss: 13.7947 - val_accuracy: 0.0195
Epoch 170/10000
89/89 [==============================] - 3s 38ms/step - loss: 1.5544 - accuracy: 0.5899 - val_loss: 13.0165 - val_accuracy: 0.0195
Epoch 171/10000
89/89 [==============================] - 3s 38ms/step - loss: 1.4613 - accuracy: 0.6127 - val_loss: 13.5445 - val_accuracy: 0.0184
Epoch 172/10000
89/89 [==============================] - 4s 41ms/step - loss: 1.4558 - accuracy: 0.6109 - val_loss: 13.5890 - val_accuracy: 0.0131
Epoch 173/10000
89/89 [==============================] - 4s 46ms/step - loss: 1.4843 - accuracy: 0.6101 - val_loss: 14.4517 - val_accuracy: 0.0180
Epoch 174/10000
89/89 [==============================] - 3s 39ms/step - loss: 1.4348 - accuracy: 0.6209 - val_loss: 13.8578 - val_accuracy: 0.0180
Epoch 175/10000
89/89 [==============================] - 4s 40ms/step - loss: 1.3919 - accuracy: 0.6346 - val_loss: 13

Epoch 225/10000
89/89 [==============================] - 3s 39ms/step - loss: 1.1039 - accuracy: 0.6942 - val_loss: 17.3122 - val_accuracy: 0.0180
Epoch 226/10000
89/89 [==============================] - 3s 39ms/step - loss: 1.0175 - accuracy: 0.7168 - val_loss: 16.5285 - val_accuracy: 0.0262
Epoch 227/10000
89/89 [==============================] - 5s 59ms/step - loss: 1.0352 - accuracy: 0.7162 - val_loss: 17.6323 - val_accuracy: 0.0184
Epoch 228/10000
89/89 [==============================] - 3s 39ms/step - loss: 0.9453 - accuracy: 0.7370 - val_loss: 17.7981 - val_accuracy: 0.0209
Epoch 229/10000
89/89 [==============================] - 3s 39ms/step - loss: 0.8933 - accuracy: 0.7494 - val_loss: 18.4905 - val_accuracy: 0.0187
Epoch 230/10000
89/89 [==============================] - 5s 51ms/step - loss: 0.9061 - accuracy: 0.7478 - val_loss: 18.1405 - val_accuracy: 0.0195
Epoch 231/10000
89/89 [==============================] - 4s 40ms/step - loss: 0.9182 - accuracy: 0.7470 - val_loss: 17

Epoch 281/10000
89/89 [==============================] - 3s 39ms/step - loss: 0.6769 - accuracy: 0.8043 - val_loss: 21.3285 - val_accuracy: 0.0184
Epoch 282/10000
89/89 [==============================] - 4s 49ms/step - loss: 0.6125 - accuracy: 0.8229 - val_loss: 21.1060 - val_accuracy: 0.0184
Epoch 283/10000
89/89 [==============================] - 4s 39ms/step - loss: 0.6244 - accuracy: 0.8219 - val_loss: 20.6561 - val_accuracy: 0.0187
Epoch 284/10000
89/89 [==============================] - 3s 39ms/step - loss: 0.6035 - accuracy: 0.8258 - val_loss: 21.3573 - val_accuracy: 0.0184
Epoch 285/10000
89/89 [==============================] - 3s 38ms/step - loss: 0.6075 - accuracy: 0.8242 - val_loss: 21.6859 - val_accuracy: 0.0177
Epoch 286/10000
89/89 [==============================] - 3s 38ms/step - loss: 0.6114 - accuracy: 0.8263 - val_loss: 21.2336 - val_accuracy: 0.0180
Epoch 287/10000
89/89 [==============================] - 4s 44ms/step - loss: 0.6031 - accuracy: 0.8237 - val_loss: 21

In [9]:
# GERA O GRAFICO DE ACURÁCIA
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{method_algo}/{timestamp}/graph_accuracy.png')
plt.close()

# GERA O GRÁFICO DE PERCA
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{method_algo}/{timestamp}/graph_loss.png')
plt.close()

In [10]:
# PEGA A MAIOR E ACURÁCIA
higher_accuracy = model.evaluate(inputs_test, targets_test, batch_size=128)

higher_accuracy = str(int(higher_accuracy[1] * 10000)).zfill(4)

28/28 [==============================] - 0s 15ms/step - loss: 4.7041 - accuracy: 0.0130


In [11]:
# RENOMEIA A PASTA
Directory.rename_directory(f'models/{method_algo}/{timestamp}',
                   f'models/{method_algo}/acc{higher_accuracy}_seed{random_state}_epochs{len(history.history["accuracy"])}_time{timestamp}')